# Textklassificeringsuppgift

I den här modulen börjar vi med en enkel textklassificeringsuppgift baserad på **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**-datasetet: vi kommer att klassificera nyhetsrubriker i en av fyra kategorier: Världen, Sport, Ekonomi och Vetenskap/Teknik.

## Datasetet

För att ladda datasetet kommer vi att använda **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**-API:et.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Vi kan nu komma åt tränings- och testdelarna av datasetet genom att använda `dataset['train']` och `dataset['test']` respektive:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Låt oss skriva ut de första 10 nya rubrikerna från vår dataset:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Textvektorisering

Nu behöver vi konvertera text till **siffror** som kan representeras som tensorer. Om vi vill ha representation på ordnivå behöver vi göra två saker:

* Använda en **tokenizer** för att dela upp texten i **tokens**.
* Bygga ett **ordförråd** av dessa tokens.

### Begränsa storleken på ordförrådet

I exemplet med AG News-datasetet är ordförrådets storlek ganska stor, över 100 000 ord. Generellt sett behöver vi inte ord som sällan förekommer i texten — bara några få meningar kommer att innehålla dem, och modellen kommer inte att lära sig något av dem. Därför är det vettigt att begränsa ordförrådets storlek till ett mindre antal genom att skicka ett argument till vektoriseringens konstruktor:

Båda dessa steg kan hanteras med hjälp av lagret **TextVectorization**. Låt oss instansiera vektoriseringsobjektet och sedan anropa metoden `adapt` för att gå igenom all text och bygga ett ordförråd:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Observera** att vi endast använder en delmängd av hela datasetet för att bygga ett ordförråd. Vi gör detta för att snabba upp exekveringstiden och för att du inte ska behöva vänta. Dock tar vi risken att vissa ord från hela datasetet inte inkluderas i ordförrådet och ignoreras under träningen. Därför bör användning av hela ordförrådsstorleken och att köra igenom hela datasetet under `adapt` öka den slutliga noggrannheten, men inte avsevärt.

Nu kan vi komma åt det faktiska ordförrådet:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Med hjälp av vektoriseringen kan vi enkelt koda vilken text som helst till en uppsättning siffror:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Bag-of-words textrepresentation

Eftersom ord representerar betydelse kan vi ibland förstå innebörden av en text genom att bara titta på de enskilda orden, oavsett deras ordning i meningen. Till exempel, när vi klassificerar nyheter, är ord som *väder* och *snö* sannolikt att indikera *väderprognos*, medan ord som *aktier* och *dollar* skulle peka mot *finansiella nyheter*.

**Bag-of-words** (BoW) vektorrepresentation är den mest lättförståeliga traditionella vektorrepresentationen. Varje ord är kopplat till ett vektorindex, och ett element i vektorn innehåller antalet förekomster av varje ord i ett givet dokument.

![Bild som visar hur en bag-of-words vektorrepresentation representeras i minnet.](../../../../../translated_images/sv/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Du kan också tänka på BoW som en summa av alla enskilt one-hot-kodade vektorer för individuella ord i texten.

Nedan är ett exempel på hur man genererar en bag-of-words representation med hjälp av Scikit Learn python-biblioteket:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Vi kan också använda Keras-vektoriseraren som vi definierade ovan, konvertera varje ordnummer till en one-hot-kodning och summera alla dessa vektorer:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Observera**: Du kanske blir förvånad över att resultatet skiljer sig från det tidigare exemplet. Anledningen är att i Keras-exemplet motsvarar längden på vektorn storleken på vokabulären, som byggdes från hela AG News-datasetet, medan vi i Scikit Learn-exemplet byggde vokabulären från exempeltexten direkt.


## Träna BoW-klassificeraren

Nu när vi har lärt oss att skapa bag-of-words-representationen av vår text, låt oss träna en klassificerare som använder den. Först måste vi konvertera vår dataset till en bag-of-words-representation. Detta kan göras genom att använda `map`-funktionen på följande sätt:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Nu ska vi definiera ett enkelt klassificeringsneuronätverk som innehåller ett linjärt lager. Indatastorleken är `vocab_size`, och utmatningsstorleken motsvarar antalet klasser (4). Eftersom vi löser en klassificeringsuppgift är den slutliga aktiveringsfunktionen **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Eftersom vi har fyra klasser är en noggrannhet över 80% ett bra resultat.

## Träna en klassificerare som ett nätverk

Eftersom vektoriseringen också är ett Keras-lager kan vi definiera ett nätverk som inkluderar det och träna det från början till slut. På så sätt behöver vi inte vektorisera datasetet med hjälp av `map`, vi kan helt enkelt skicka det ursprungliga datasetet till nätverkets input.

> **Note**: Vi måste fortfarande använda `map` på vårt dataset för att konvertera fält från ordböcker (såsom `title`, `description` och `label`) till tupler. Men när vi laddar data från disk kan vi bygga ett dataset med den nödvändiga strukturen direkt från början.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammer, trigrammer och n-gram

En begränsning med bag-of-words-metoden är att vissa ord ingår i flerordsuttryck. Till exempel har ordet 'hot dog' en helt annan betydelse än orden 'hot' och 'dog' i andra sammanhang. Om vi alltid representerar orden 'hot' och 'dog' med samma vektorer kan det förvirra vår modell.

För att hantera detta används ofta **n-gram-representationer** i metoder för dokumentklassificering, där frekvensen av varje ord, tvåords- eller treordsuttryck är en användbar egenskap för att träna klassificerare. I bigram-representationer, till exempel, lägger vi till alla ordpar i vokabulären, utöver de ursprungliga orden.

Nedan är ett exempel på hur man genererar en bigram bag-of-words-representation med hjälp av Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Den största nackdelen med n-gram-metoden är att vokabulärstorleken börjar växa extremt snabbt. I praktiken behöver vi kombinera n-gram-representationen med en teknik för dimensionsreduktion, såsom *embeddings*, vilket vi kommer att diskutera i nästa enhet.

För att använda en n-gram-representation i vårt **AG News**-dataset, behöver vi skicka `ngrams`-parametern till vår `TextVectorization`-konstruktor. Längden på ett bigram-vokabulär är **avsevärt större**, i vårt fall är det mer än 1,3 miljoner tokens! Därför är det rimligt att begränsa bigram-tokens till ett rimligt antal.

Vi skulle kunna använda samma kod som ovan för att träna klassificeraren, men det skulle vara väldigt minnesineffektivt. I nästa enhet kommer vi att träna bigram-klassificeraren med hjälp av embeddings. Under tiden kan du experimentera med att träna bigram-klassificeraren i denna notebook och se om du kan uppnå högre noggrannhet.


## Automatisk beräkning av BoW-vektorer

I exemplet ovan beräknade vi BoW-vektorer manuellt genom att summera one-hot-enkodningarna av enskilda ord. Den senaste versionen av TensorFlow gör det dock möjligt att automatiskt beräkna BoW-vektorer genom att ange parametern `output_mode='count` till vektoriseringens konstruktor. Detta gör det betydligt enklare att definiera och träna vår modell:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Termfrekvens - invers dokumentfrekvens (TF-IDF)

I BoW-representationen viktas ordens förekomster med samma teknik oavsett själva ordet. Det är dock tydligt att frekventa ord som *en* och *i* är mycket mindre viktiga för klassificering än specialiserade termer. I de flesta NLP-uppgifter är vissa ord mer relevanta än andra.

**TF-IDF** står för **termfrekvens - invers dokumentfrekvens**. Det är en variation av bag-of-words, där man istället för ett binärt 0/1-värde som indikerar förekomsten av ett ord i ett dokument, använder ett flyttalsvärde som relaterar till frekvensen av ordets förekomst i korpusen.

Mer formellt definieras vikten $w_{ij}$ för ett ord $i$ i dokumentet $j$ som:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
där
* $tf_{ij}$ är antalet förekomster av $i$ i $j$, dvs. BoW-värdet vi har sett tidigare
* $N$ är antalet dokument i samlingen
* $df_i$ är antalet dokument som innehåller ordet $i$ i hela samlingen

TF-IDF-värdet $w_{ij}$ ökar proportionellt med antalet gånger ett ord förekommer i ett dokument och justeras med antalet dokument i korpusen som innehåller ordet, vilket hjälper till att kompensera för att vissa ord förekommer oftare än andra. Till exempel, om ordet förekommer i *varje* dokument i samlingen, $df_i=N$, och $w_{ij}=0$, och dessa termer skulle helt ignoreras.

Du kan enkelt skapa TF-IDF-vektorisering av text med Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

I Keras kan `TextVectorization`-lagret automatiskt beräkna TF-IDF-frekvenser genom att ange parametern `output_mode='tf-idf'`. Låt oss upprepa koden vi använde ovan för att se om användning av TF-IDF ökar noggrannheten:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Slutsats

Även om TF-IDF-representationer tilldelar frekvensvikter till olika ord, kan de inte representera betydelse eller ordning. Som den berömda lingvisten J. R. Firth sa år 1935: "Den fullständiga betydelsen av ett ord är alltid kontextuell, och ingen studie av betydelse utanför kontext kan tas på allvar." Vi kommer senare i kursen att lära oss hur man fångar kontextuell information från text med hjälp av språkmodellering.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
